In [37]:
import numpy as np
import scipy.interpolate as spi
import scipy.sparse as sp
import scipy.linalg as sla

In [38]:
def ex(S, K, r, q, sigma, T, Smin, Smax, m, n, type):
    omega = 1 if type == 'Call' else -1
    dS = (Smax-Smin)/m
    dt = T/n
    Svec = np.linspace(Smin, Smax, m+1)
    Tvec = np.linspace(0, T, n+1)
    grid = np.zeros(shape=(m+1, n+1))

    # terminal
    grid[:, -1] = np.maximum(omega*(Svec - K), 0)

    # boundary
    tau = Tvec[-1] - Tvec;     
    DFq = np.exp(q * tau)
    DFr = np.exp(r * tau)
    # american option boundary condition should differ from Euro style
    grid[0,  :] = np.maximum(omega*(Svec[0]  - K), 0)
    grid[-1, :] = np.maximum(omega*(Svec[-1] - K), 0) 

    # set coefficient
    drift = (r-q) * Svec[1:-1]/dS
    diffusion_square = (sigma*Svec[1:-1]/dS)**2

    l = 0.5*(diffusion_square - drift)
    c = -diffusion_square - r
    u = 0.5*(diffusion_square + drift)

    # set matrix (expl)
    A = sp.diags([l[1:], c, u[:-1]], [-1, 0, 1],  format='csc')
    I = sp.eye(m-1)
    M = I + dt*A

    # solve (expl)
    for j in reversed(np.arange(n)):
        U = M.dot(grid[1:-1, j+1])
        # print(U)
        U[0] += l[0]*dt*grid[0, j+1] 
        U[-1] += u[-1]*dt*grid[-1, j+1] 
        grid[1:-1, j] = U

    # interpolate
    tck = spi.splrep( Svec, grid[:,0], k=3 )
    res = spi.splev( S, tck )
    return res

In [39]:
def im(S, K, r, q, sigma, T, Smin, Smax, m, n, type):
    omega = 1 if type == 'Call' else -1
    dS = (Smax-Smin)/m
    dt = T/n
    Svec = np.linspace(Smin, Smax, m+1)
    Tvec = np.linspace(0, T, n+1)
    grid = np.zeros(shape=(m+1, n+1))
    # terminal
    grid[:, -1] = np.maximum(omega*(Svec - K), 0)
    # boundary
    tau = Tvec[-1] - Tvec;     
    DFq = np.exp(q * tau)
    DFr = np.exp(r * tau)
    # american option boundary condition should differ from Euro style
    grid[0,  :] = np.maximum(omega*(Svec[0]  - K), 0)
    grid[-1, :] = np.maximum(omega*(Svec[-1] - K), 0) 
    # set coefficient
    drift = (r-q) * Svec[1:-1]/dS
    diffusion_square = (sigma*Svec[1:-1]/dS)**2
    l = 0.5*(diffusion_square - drift)
    c = -diffusion_square - r
    u = 0.5*(diffusion_square + drift)
    # set matrix (imp)
    A = sp.diags([l[1:], c, u[:-1]], [-1, 0, 1],  format='csc')
    I = sp.eye(m-1)
    M = I - dt * A
    # solve (imp)
    _, M_lower, M_upper = sla.lu(M.toarray())
    for j in reversed(np.arange(n)):      
        U = grid[1:-1, j+1].copy()
        U[0] += l[0]*dt*grid[0, j] 
        U[-1] += u[-1]*dt*grid[-1, j] 
        Ux = sla.solve_triangular( M_lower, U, lower=True )
        grid[1:-1, j] = sla.solve_triangular( M_upper, Ux, lower=False )
    # interpolate
    tck = spi.splrep( Svec, grid[:,0], k=3 )
    res = spi.splev( S, tck )
    return res

In [40]:
S = 40
K = 40
r = 0.01
q = 0.005
sigma = 0.2
T = 1
Smin = 2
Smax = 80
m = 500 # Ns (number of partitions for the stock price)
n = 500 # Nt (number of partitions for the time to maturity).


In [41]:
type = 'Put'
print(f'implicit finite difference methods for {type} : {im(S, K, r, q, sigma, T, Smin, Smax, m, n, type)}')

implicit finite difference methods for Put : 3.063341212310613


In [42]:
type = 'Call'
print(f'implicit finite difference methods for {type} : {im(S, K, r, q, sigma, T, Smin, Smax, m, n, type)}')

implicit finite difference methods for Call : 3.261844075743102
